<a href="https://colab.research.google.com/github/caiocezar400/DeepSea-Solutions/blob/main/Trajet%C3%B3ria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "numpy<2.0" roboticstoolbox-python spatialmath-python --force-reinstall
!pip install PyQt5

In [ ]:
%matplotlib inline

import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys

# -----------------------------------------------------------------------------
# 1. FUNÇÕES DE ENTRADA ROBUSTA
# -----------------------------------------------------------------------------
def input_float(prompt, default_value):
    """Solicita float com tratamento de erro e valor padrão."""
    try:
        val_str = input(f"{prompt} [Padrão: {default_value:.3f}]: ").strip()
        if not val_str: return float(default_value)
        return float(val_str)
    except ValueError:
        print(f"  -> Entrada inválida. Usando {default_value}")
        return float(default_value)

def input_joint_target(robot, name="Alvo"):
    """
    Solicita a configuração de juntas ao usuário.
    Essencial para o Modo 2 (Juntas) funcionar interativamente.
    """
    print(f"\n--- Definir Juntas ({name}) ---")
    q_out = []
    for i, link in enumerate(robot.links):
        curr = robot.qn[i]
        if link.isprismatic:
            val = input_float(f"  Junta {i+1} (Prismática) [m]", curr)
            q_out.append(val)
        else:
            curr_deg = np.rad2deg(curr)
            val_deg = input_float(f"  Junta {i+1} (Revoluta) [graus]", curr_deg)
            q_out.append(np.deg2rad(val_deg))
    return np.array(q_out)

# -----------------------------------------------------------------------------
# 2. CONSTRUÇÃO DO ROBÔ (DH PARAMETERS)
# -----------------------------------------------------------------------------

def criar_robo_prebuilt():
    """
    Retorna um robô RRRP (SCARA Estendido) configurado automaticamente.
    """
    print("\n--- Carregando Robô Pré-construído: RRRP (SCARA Estendido) ---")
    print("Estrutura: Base (R) -> Cotovelo (R) -> Punho (R) -> Atuador Z (P)")

    # Definição dos elos usando a classe DHRobot do toolbox
    links = [
        # Junta 1 (R)
        rtb.RevoluteDH(d=0.3, a=0.5, alpha=0, qlim=[-np.pi*0.8, np.pi*0.8]),

        # Junta 2 (R)
        rtb.RevoluteDH(d=0.0, a=0.5, alpha=0, qlim=[-np.pi*0.8, np.pi*0.8]),

        # Junta 3 (R)
        # Alpha=pi inverte o eixo Z para apontar para baixo
        rtb.RevoluteDH(d=0.0, a=0.2, alpha=np.pi, qlim=[-np.pi, np.pi]),

        # Junta 4 (P)
        rtb.PrismaticDH(theta=0, a=0, alpha=0, qlim=[0.0, 0.5])
    ]

    robot = rtb.DHRobot(links, name="SCARA_RRRP")

    # Define pose 'Home' (qn) segura
    robot.qn = np.array([0, 0, 0, 0.1])

    return robot

def criar_robo_manual():
    """
    Solicita os parâmetros DH ao usuário.
    """
    print("\n=== Configuração Manual do Manipulador ===")
    print("Sugestões: 'RRRP' (SCARA), 'RRRRRR' (Antropomórfico 6-Eixos)")
    config_str = input("Digite a sequência de elos: ").strip().upper()
    if not config_str: config_str = "RRRP"

    links = []
    print(f"\nDefinindo Parâmetros DH para {config_str}...")

    for i, tipo in enumerate(config_str):
        print(f"\n--- Elo {i+1} ({tipo}) ---")
        if tipo == 'R':
            d = input_float("  d (offset Z)", 0.0)
            a = input_float("  a (comprimento X)", 0.5 if i==0 else 0.4)
            alpha_deg = input_float("  alpha (torção X em graus)", 0.0)
            links.append(rtb.RevoluteDH(d=d, a=a, alpha=np.deg2rad(alpha_deg), qlim=[-np.pi, np.pi]))

        elif tipo == 'P':
            theta_deg = input_float("  theta (rotação Z em graus)", 0.0)
            a = input_float("  a (comprimento X)", 0.0)
            alpha_deg = input_float("  alpha (torção X em graus)", 0.0)
            lim_max = input_float("  limite max (m)", 0.5)
            links.append(rtb.PrismaticDH(theta=np.deg2rad(theta_deg), a=a, alpha=np.deg2rad(alpha_deg), qlim=[0.0, lim_max]))

    if not links: sys.exit(1)

    r = rtb.DHRobot(links, name=f"Custom_{config_str}")

    # Define pose Home segura (qn) baseada nos limites
    q_nominal = np.zeros(r.n)
    for i, link in enumerate(links):
        if link.isprismatic:
            q_nominal[i] = (link.qlim[0] + link.qlim[1]) / 2.0
    r.qn = q_nominal

    return r

def selecionar_robo():
    print("========================================")
    print("   SIMULADOR ROBOTICS TOOLBOX (RTB)")
    print("========================================")
    print("1. Robô Pré-construído (SCARA RRRP)")
    print("2. Configuração Manual")

    op = input("Opção [1]: ").strip() or '1'

    if op == '1':
        return criar_robo_prebuilt()
    else:
        return criar_robo_manual()

# -----------------------------------------------------------------------------
# 3. GERAÇÃO DE TRAJETÓRIA
# -----------------------------------------------------------------------------
def get_trajectory(traj_type, center_pose, scale, steps):

    cx, cy, cz = center_pose.t
    R = center_pose.R # Matriz de Rotação original
    T = []

    if traj_type == 'circle':
        ts = np.linspace(0, 2*np.pi, steps)
        for t in ts:
            x = cx + scale * np.cos(t)
            y = cy + scale * np.sin(t)
            T.append(sm.SE3(x, y, cz) * sm.SE3(R))

    elif traj_type == 'semicircle':
        # Arco de 180 graus (Pi radianos)
        ts = np.linspace(0, np.pi, steps)
        for t in ts:
            x = cx + scale * np.cos(t)
            y = cy + scale * np.sin(t)
            T.append(sm.SE3(x, y, cz) * sm.SE3(R))

    elif traj_type == 'line':
        ys = np.linspace(-scale, scale, steps)
        for y_off in ys:
            T.append(sm.SE3(cx, cy + y_off, cz) * sm.SE3(R))

    elif traj_type == 'figure8':
        ts = np.linspace(0, 2*np.pi, steps)
        for t in ts:
            x = cx + scale * np.sin(t)
            y = cy + scale * np.sin(t) * np.cos(t)
            T.append(sm.SE3(x, y, cz) * sm.SE3(R))

    return sm.SE3(T)

# -----------------------------------------------------------------------------
# 4. VISUALIZADOR MANUAL
# -----------------------------------------------------------------------------
def plot_manual_robot(robot, q_config, ax, color='blue', alpha=1.0):
    """Calcula FK manualmente e plota linhas usando Matplotlib puro."""
    T = sm.SE3()
    points = [T.t]

    for link, q in zip(robot.links, q_config):
        T = T * link.A(q)
        points.append(T.t)

    points = np.array(points)

    # Plota estrutura
    ax.plot(points[:, 0], points[:, 1], points[:, 2],
            '-o', color=color, linewidth=2, alpha=alpha, markersize=4)

    # Plota efetuador
    if alpha > 0.8:
        ax.scatter(points[-1, 0], points[-1, 1], points[-1, 2],
                   color='red', s=50, marker='*', zorder=10)

    return T

# -----------------------------------------------------------------------------
# 5. LÓGICA PRINCIPAL
# -----------------------------------------------------------------------------
def main():
    # 1. Configurar Robô (Seleção)
    robot = selecionar_robo()
    print(f"\nRobô criado: {robot.name}")
    print(robot)

    # 2. Configurar Matplotlib
    plt.ion() # Modo interativo ON
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Calcula limites do gráfico automaticamente baseado no tamanho do robô
    limit = sum([abs(l.a) for l in robot.links]) + sum([abs(l.d) for l in robot.links]) + 0.5

    def reset_plot_limits():
        ax.set_xlim(-limit, limit)
        ax.set_ylim(-limit, limit)
        ax.set_zlim(0, limit*1.2)
        ax.set_xlabel('X [m]')
        ax.set_ylabel('Y [m]')
        ax.set_zlabel('Z [m]')
        ax.set_title(f"Simulação: {robot.name}")

    reset_plot_limits()

    # 3. Escolha do Modo
    print("\nEscolha o Modo de Simulação:")
    print("1 - Seguir Trajetória (Inverse Kinematics)")
    print("2 - Mover para Juntas (Forward Kinematics)")
    mode = input("Opção [1/2]: ").strip()

    if mode == '2':
        # --- MODO FK ---
        while True:
            ax.cla() # Limpa para redesenhar
            reset_plot_limits()

            # Desenha Home (transparente)
            plot_manual_robot(robot, robot.qn, ax, color='gray', alpha=0.3)

            # Pede e desenha Target
            q_target = input_joint_target(robot, "Nova Pose")
            T_end = plot_manual_robot(robot, q_target, ax, color='blue')
            print(f"\nPose Final:\n{T_end}")

            plt.draw()
            plt.pause(0.1)
            if input("Definir outra pose? (s/n): ").lower() != 's': break

    else:
        # --- MODO IK (TRAJETÓRIA) ---
        print("\nConfiguração da Trajetória:")
        print("Opções: [circle, semicircle, line, figure8]")
        type_traj = input("Tipo: ").strip().lower()
        if type_traj not in ['circle', 'semicircle', 'line', 'figure8']:
            print("Padrão: circle")
            type_traj = 'circle'

        # Ponto central da trajetória
        T_home = robot.fkine(robot.qn)

        # Desloca um pouco à frente (X) para não colidir com a base
        center_pose = T_home * sm.SE3(0.2, 0, 0)

        scale = input_float("Raio/Tamanho [m]", 0.2)
        steps = int(input_float("Passos (suavidade)", 40))

        traj_SE3 = get_trajectory(type_traj, center_pose, scale, steps)
        q_current = robot.qn

        print(f"\nIniciando animação ({len(traj_SE3)} frames)...")

        for i, T_desejado in enumerate(traj_SE3):
            ax.cla()
            reset_plot_limits()

            # --- SOLUÇÃO IK COM MÁSCARA ---
            # mask=[1,1,1,0,0,0] -> Resolve XYZ, ignora rotação (Roll, Pitch, Yaw)
            # Crucial para robôs com < 6 eixos (como o SCARA 4-Eixos)
            ik_sol = robot.ikine_LM(T_desejado, q0=q_current, mask=[1, 1, 1, 0, 0, 0])

            # Desenha o alvo (ponto verde)
            ax.scatter(T_desejado.t[0], T_desejado.t[1], T_desejado.t[2], c='green', marker='.', s=15)

            if ik_sol.success:
                q_current = ik_sol.q
                # Desenha o robô com 'rastro' (alpha 0.4)
                plot_manual_robot(robot, q_current, ax, color='blue', alpha=0.5)
            else:
                print(f"Frame {i}: Falha na IK (Singularidade ou Fora do Alcance)")

            plt.draw()
            plt.pause(0.05)

        print("\nTrajetória concluída.")
        plt.ioff()
        plt.show()

if __name__ == "__main__":
    main()